# Live testing

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
from datetime import datetime, timedelta

import ectools as ec

script_dir = os.path.dirname(os.path.abspath('testing'))
FPATH = os.path.normpath(os.path.join(script_dir, 'data/'))
FNAME = '241007_01_MCL21_cswWE1_CV2_CO2_750C.DTA'

imp = ec.EcImporter()
f = imp.load_file(FPATH, FNAME)

In [2]:
fl = imp.load_folder(FPATH)

ectools.parse_file_gamry error:
'utf-8' codec can't decode byte 0xb0 in position 7033: invalid start byte
Processed 53 files, parsed 52


In [3]:
f = fl[2]

f.curr.size

f.ge

2578

In [4]:
a = np.array(f.time, dtype=timedelta)[10]

In [8]:
from typing import List
lst = [1, '2', 3, 4, 5, 6, 7, 8, 9, 10] # type: List[str]

a = lst[3]
print(type(a))

b = lst.__getitem__(2)
print(b)

<class 'int'>
3


In [1]:
# File select

# Integration tool v 1.0
from bokeh.models import ColumnDataSource, CustomJS, BoxSelectTool, Div, Button, DataTable, TableColumn
from bokeh.plotting import figure, show
from bokeh.layouts import column, row
from bokeh.io import output_notebook
from bokeh.palettes import Category10

output_notebook()

# Initialize data sources
source = ColumnDataSource(f.get_data_dict())
selected_area_source = ColumnDataSource(data=dict(xs=[], ys=[], color=[]))
current_selection_source = ColumnDataSource(data=dict(x=[], y=[]))
integration_results_source = ColumnDataSource(data=dict(area=[]))

# Create plot
plot = figure(x_axis_label='Time', y_axis_label='Current', title="Select Integration Range")
l = plot.line('time', 'curr', source=source, line_width=2, color="navy")
r = plot.scatter('time', 'curr', source=source, size=1, color="navy", alpha=0)

# Add tools
hover = f.get_hover_tool()
hover.renderers = [l]
box_select = BoxSelectTool(renderers=[r], dimensions='width')
box_select.continuous = True
plot.add_tools(box_select, hover)
plot.toolbar.active_drag = box_select

# Add patch glyphs
plot.patches('xs', 'ys', source=selected_area_source, color='color', alpha=0.5)
plot.patch('x', 'y', source=current_selection_source, color='green', alpha=0.5)

# Create a Div to display integration results
result_div = Div(text="")

# Create a button to add the integral result
add_button = Button(label="Add Integral", button_type="success")

# Create a table to display integration results
columns = [TableColumn(field="area", title="Integrated Area")]
data_table = DataTable(source=integration_results_source, columns=columns, width=400, height=280)

# JavaScript callback to perform integration and update the current selection patch
callback = CustomJS(args=dict(source=source, current_selection_source=current_selection_source, result_div=result_div), code="""
    var data = source.data;
    var selected = source.selected.indices;
    var x = data['time'];
    var y = data['curr'];

    // Sort selected indices
    selected.sort(function(a, b) { return x[a] - x[b]; });

    // Extract selected data
    var selected_x = [];
    var selected_y = [];
    for (var i = 0; i < selected.length; i++) {
        selected_x.push(x[selected[i]]);
        selected_y.push(y[selected[i]]);
    }

    // Add the first and last x values with y=0 to close the area
    if (selected_x.length > 0) {
        selected_x.unshift(selected_x[0]);
        selected_y.unshift(0);
        selected_x.push(selected_x[selected_x.length - 1]);
        selected_y.push(0);
    }

    // Update the current selection source
    current_selection_source.data = { x: selected_x, y: selected_y };
    current_selection_source.change.emit();

    // Calculate the area under the curve using the trapezoidal rule
    var area = 0;
    for (var i = 0; i < selected_x.length - 1; i++) {
        area += (selected_x[i + 1] - selected_x[i]) * (selected_y[i] + selected_y[i + 1]) / 2;
    }

    // Update the result div to display the current integration result
    result_div.text = "Current Integrated Area: " + area.toFixed(2);
""")
source.selected.js_on_change('indices', callback)

# JavaScript callback to add the integral result to the table and save the patch
add_callback = CustomJS(args=dict(current_selection_source=current_selection_source, selected_area_source=selected_area_source, result_div=result_div, integration_results_source=integration_results_source), code="""
    var area_text = result_div.text.split(": ")[1];
    var area = parseFloat(area_text);
    var data = integration_results_source.data;
    data['area'].push(area.toFixed(2));
    integration_results_source.change.emit();

    // Append the current selection to the existing data
    var existing_xs = selected_area_source.data['xs'];
    var existing_ys = selected_area_source.data['ys'];
    var existing_color = selected_area_source.data['color'];

    var selected_x = current_selection_source.data['x'];
    var selected_y = current_selection_source.data['y'];

    // Generate a new color for the patch
    var colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'];
    var new_color = colors[existing_color.length % colors.length];

    existing_xs.push(selected_x.slice());
    existing_ys.push(selected_y.slice());
    existing_color.push(new_color);

    selected_area_source.data = { 
        xs: existing_xs, 
        ys: existing_ys,
        color: existing_color
    };
    selected_area_source.change.emit();

    // Clear the current selection
    current_selection_source.data = { x: [], y: [] };
    current_selection_source.change.emit();
""")
add_button.js_on_click(add_callback)

# Display the plot, button, and table
show(column(plot, result_div, add_button, data_table), notebook_handle=True)

Loading BokehJS ...

NameError: name 'f' is not defined

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import streams
import panel as pn

hv.extension('bokeh')
pn.extension()

# Simulate data (replace this with your real data)
time = np.linspace(0, 10, 500)
current = np.sin(time) + np.random.normal(0, 0.1, len(time))
data = pd.DataFrame({'time': time, 'current': current})

# Create Holoviews plot with BoxSelectTool enabled
curve = hv.Curve((data['time'], data['current']), 'Time', 'Current').opts(
    width=600, height=400, tools=['hover', 'box_select'], active_tools=['box_select']
)

# Stream for capturing BoxSelectTool selections
box_stream = streams.Selection1D(source=curve)

# DataFrame to hold the integration results
integration_df = pd.DataFrame(columns=['Index', 'Area', 'Color'])

# Panel components
result_table = pn.widgets.DataFrame(integration_df, width=300, height=200)
result_div = pn.pane.Markdown("### Select an area to integrate", width=300)
add_button = pn.widgets.Button(name="Add Integral", button_type='success')

# Colors for each selection
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
selected_color_idx = 0

# Function to perform the integration on selection
def integrate_selection(event):
    global selected_color_idx
    selected = box_stream.index  # Get the selected indices
    if selected:
        selected_data = data.iloc[selected]
        area = np.trapz(selected_data['current'], selected_data['time'])
        
        # Update integration results table
        new_entry = pd.DataFrame({
            'Index': [len(integration_df) + 1],
            'Area': [area],
            'Color': [colors[selected_color_idx % len(colors)]]
        })
        integration_df.loc[len(integration_df)] = new_entry.iloc[0]
        result_table.value = integration_df
        
        # Highlight the selected area on the plot
        selected_patch = hv.Area((selected_data['time'], selected_data['current'])).opts(
            fill_color=colors[selected_color_idx % len(colors)], fill_alpha=0.5
        )
        result_div.object = f"Current Integrated Area: {area:.2f}"
        
        # Increment color index for next selection
        selected_color_idx += 1
        
        # Update plot
        layout[1] = curve * selected_patch

# Attach the callback to the button
add_button.on_click(integrate_selection)

# Create Panel layout
layout = pn.Row(
    pn.Column(result_div, add_button, result_table),
    curve
)

# Display the Panel dashboard in Jupyter
layout
